In [2]:
import pandas as pd
import csv

In [3]:
csv_path = 'y37_rebar.csv'
# csv_path = 'cq881_rebar.csv'

# 定義自定義解析器
def custom_csv_parser(file_path, encoding='Big5'):
    rows = []
    with open(file_path, encoding=encoding) as file:
        reader = csv.reader(file)
        for row in reader:
            if len(row) > 8:
                row[7] = ', '.join(row[7:])
                row = row[:8]
            row[-1] = row[-1].rstrip(', ')
            rows.append(row)
    return rows


# 使用自定義解析器讀取文件
rows = custom_csv_parser(csv_path)

# 定義標題列
columns = ["FileName", "EntityName", "ObjectType", "RotationAngle", "CentreCoor", "Height", "Width", "Text"]

# 將結果轉換為 DataFrame
df = pd.DataFrame(rows[1:], columns=columns)

In [26]:
rows, columns = df.shape
for i in range(rows):
    print(df.iloc[i]['Text'])


SE 183~SE 186
及                           。
SE 181~SE 182
SE 001 ,  002 .
SE 001 ,  002
SEE  DRG  SE 181~SE 182  AND  SE 183~SE 186.
OR  THE  SUGGESTED  SEQUENCE  OF  OPERATION ,  HE
SHALL  RE-DESIGN  ALL  OF  THE  ABOVE  LISTED
ITEMS  AND  RESUBMIT  THE  REVISED  DESIGN  TO  THE
ENGINEER  FOR  APPROVAL .
CONTRACTOR  WISHES  TO  CHANGE  THE  STRUT  LEVELS
TO  THE  COMMENCEMENT  OF  CONSTRUCTION . IF  THE
PROCEDURES  TO  THE  ENGINEER  FOR  APPROVAL  PRIOR
CALCULATIONS AND  STATEMENT  OF  CONSTRUCTION
THE  SPECIFIED  LOADS  AND  SHALL  SUBMIT  PLANS
AND  DESIGN  THE  STRUTTING  SYSTEM  TO  WITHSTAND
THE  DETAILS  REGARDING  HIS  CONSTRUCTION  METHODS
SATISFY  HIMSELF   REGARDING  THE  ADEQUACY  OF
SERVE  AS  A  REFERENCE .  THE  CONTRACTOR  SHALL
DETAILS  SHOWN  ON  THE  DRAWING  ARE  GIVEN  TO
RETAINING  WALL  ,   STRUT  LOCATIONS  AND  OTHER
SHOULD  NOT  BE  VIBRATIONAL.
AND  THE  CONSTRUCTION  METHOD  OF  THE  PILE
AREA  UNLESS  APPROVED  BY  THE  ENGINEER
道工程施工範圍且中間樁施工方式須為無振動工法。
AVO

In [4]:
import re
import pandas as pd

grouped = df.groupby('FileName')

# 初始化一個集合，用於存儲鋼板樁文字(未經處理)
sheet_pile_text: set = set()
# 初始劃一個字典，用於存儲鋼板樁文字(經過處理)
sheet_pile_text_dict: dict = {}

for name, group in grouped:
    print(f'File Name: {name}')

    pattern:str=  r"(\S+)\s+SHEET\s+PILE\s+\(L=(\d+)m\)"

    group['Type_name'], group['Length'] = zip(*group['Text'].str.extract(pattern).values)
    group['Length'] = pd.to_numeric(group['Length'], errors='coerce')
    group['Length'] = group['Length'].astype('Int64')

    # get all the sheet pile text without Nan
    sheet_pile_rows: pd.DataFrame = group.dropna(subset=['Type_name', 'Length']).copy()
    # combine the 'Type_name' and 'Length' columns
    sheet_pile_rows['Full_name'] = sheet_pile_rows['Type_name'] + ' ' + sheet_pile_rows['Length'].astype(str) + 'm'
    # add the 'Full_name' to the set
    sheet_pile_text.update(sheet_pile_rows['Full_name'])

print(sheet_pile_text)
# extract the sheet pile text from the set and add it to the dictionary
for text in sheet_pile_text:
    pattern = r"(\S+)\s+(\d+)m"
    name, length = re.match(pattern, text).groups()
    # sheet_pile_text_dict[text] = int(length)
    sheet_pile_text_dict[text] = {'name': name, 'length': int(length)}

print(sheet_pile_text_dict)


File Name: F752SE1750.dwg
{'FSP-Ⅲ 13m', 'FSP-Ⅲ 21m'}
{'FSP-Ⅲ 13m': {'name': 'FSP-Ⅲ', 'length': 13}, 'FSP-Ⅲ 21m': {'name': 'FSP-Ⅲ', 'length': 21}}
